In [120]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import util.framediff_utils as du
import os
import pathlib
import dgl
import copy
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
import logging
from collections import defaultdict
torch.cuda.is_available()

True

In [121]:
from gudiff_model.Graph_UNet_Null import  GraphUNet_Null
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph_Null import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph_Null import  Make_nullKNN_MP_Graphs
from data_rigid_diffuser.diffuser import FrameDiffNoise
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.FAPE_Loss import FAPE_loss_null, FAPE_loss_real
from se3_transformer.model.fiber import Fiber
from se3_transformer.runtime.utils import str2bool, to_cuda

In [122]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

# #find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')

In [123]:
from datetime import datetime

In [124]:
import torch.utils.data as data_utils

In [140]:
class Experiment:

    def __init__(self,
                 config_path= 'test.config',
                 config_path_diffuser = 'data_rigid_diffuser/base.yaml',
                 limit=5028,
                 ckpt_model=None,
                name='gu_null',
                ckpt_opt=None):
        """Initialize experiment.
        Args:
            exp_cfg: Experiment configuration.
        """
#         with open(config_path, 'r') as file:
#             config = yaml.safe_load(file)
#         conf = Struct(config)
        
        logging.basicConfig(filename='test.log', level=logging.INFO)
        self._log = logging.getLogger(__name__)
        
        
        #indices for, unsure if needed
        self.CA = Data_Graph.CA
        self.N = Data_Graph.N
        self.C = Data_Graph.C

        self.name=name
        
        config_path='data_rigid_diffuser/base.yaml'
        fnd = FrameDiffNoise(config_path)
        
        conf = {'batch_size'  : 8,
                      'topk'  : 4,
                    'stride'  : 4,
                        'KNN' : 30,
                  'num_heads' : 16,
                   'channels' : 64,
               'channels_div' : 8,
                 'num_layers' : 1,
             'num_layers_ca'  : 2,
           'edge_feature_dim' : 1,
          'latent_pool_type'  : 'avg',
                    't_size'  : 12,
                       'mult' : 2,
                   'zero_lin' : True,
                  'use_tdeg1' : False,
                        'cuda': True,
              'learning rate' : 0.005,
               'weight_decay' : 0.000001,
                'sc_nf_real'  : 0.2 ,
                'sc_3D_real'  : 1.0 ,
                'sc_3D_null'  : 1.0 ,
                'device'      : 'cuda',
                'num_epoch'   : 100,
                'log_freq'    : 50,
                'ckpt_freq'   : 1000,
                'early_chkpt' : 10,
                'coord_scale' : 10.0
               }
        
        

        
        conf_check = {'ckpt_dir' : 'GUN_checkpoints/',
                      'eval_dir' : 'Eval_Direc',
                      'apa_path' : 'data_npose/h4_apa_coords.npz',
                      'tog_path' : 'data_npose/h4_tog_coords.npz',
                     }
        self._conf = conf
        

        
        self.device = conf['device']
        self.paths_valid = conf_check['apa_path']
        self.paths_train = conf_check['tog_path']
        self.score_weights = {}
        self.score_weights['nf_real'] = torch.tensor(float(conf['sc_nf_real']),device=self.device)
        self.score_weights['3D_real'] = torch.tensor(float(conf['sc_3D_real']),device=self.device)
        self.score_weights['3D_null'] = torch.tensor(float(conf['sc_3D_null']),device=self.device)

        #needed
        self.coord_scale = conf['coord_scale']
        self.N_CA_dist = (Data_Graph.N_CA_dist/self.coord_scale).to('cuda')
        self.C_CA_dist = (Data_Graph.C_CA_dist/self.coord_scale).to('cuda')
        
        self.num_epoch = conf['num_epoch']
        self.log_freq = conf['log_freq']
        self.ckpt_freq = conf['ckpt_freq']
        self.early_ckpt = conf['early_chkpt']
        
        self.B = conf['batch_size']
        self.L = 128
        self.limit = limit
        self.KNN = conf['KNN']
        self.stride = conf['stride']
        
        
        
        self.fnd = FrameDiffNoise(config_path_diffuser)
        self.mkg = Make_nullKNN_MP_Graphs(KNN = self.KNN, mp_stride = self.stride, n_nodes = self.L)
        self._model = GraphUNet_Null(fiber_start = Fiber({0:17, 1:2}),
                                     fiber_out = Fiber({0:5,1:2}),
                                          k = conf['topk'],
                                batch_size  = self.B,
                                     stride = conf['stride'],
                                 max_degree = 3,
                                   channels = conf['channels'],
                                  num_heads = conf['num_heads'],
                               channels_div = conf['channels_div'],
                                 num_layers = conf['num_layers'],
                              num_layers_ca = conf['num_layers_ca'],
                           edge_feature_dim = conf['edge_feature_dim'],
                           latent_pool_type = conf['latent_pool_type'],
                                     t_size = conf['t_size'],
                                       mult = conf['mult'],
                                   zero_lin = conf['zero_lin'],
                                  use_tdeg1 = conf['use_tdeg1'],
                                        cuda= conf['cuda'])
        
        
        num_parameters = sum(p.numel() for p in self._model.parameters())
        self.num_parameters = num_parameters
        self._log.info(f'Number of model parameters {num_parameters}')
#         self._optimizer = EMA(0.980)
#         for name, param in self._model.named_parameters():
#             if param.requires_grad:
#                 self._optimizer.register(name, param.data)

        if ckpt_model is not None:
            ckpt_model = {k.replace('module.', ''):v for k,v in ckpt_model.items()}
            self._model.load_state_dict(ckpt_model, strict=True)
        
        
        self._optimizer = torch.optim.Adam( self._model.parameters(),
                                                       lr=conf['learning rate'],
                                                       weight_decay=conf['weight_decay'])
        if ckpt_opt is not None:
            self._optimizer.load_state_dict(ckpt_opt)
        
        
        dt_string = datetime.now().strftime("%dD_%mM_%YY_%Hh_%Mm_%Ss")
        self.ckpt_dir =  conf_check['ckpt_dir']
        self.eval_dir = conf_check['eval_dir']
        if self.ckpt_dir is not None:
            # Set-up checkpoint location
            ckpt_dir = os.path.join(
                 self.ckpt_dir,
                 self.name,
                 dt_string)
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir, exist_ok=True)
            self.ckpt_dir = ckpt_dir
            self._log.info(f'Checkpoints saved to: {ckpt_dir}')
        else:  
            self._log.info('Checkpoint not being saved.')
            
        if self.eval_dir is not None :
            self.eval_dir = os.path.join(
                self.eval_dir,
                self.name,
                dt_string)
            self.eval_dir = self.eval_dir
            self._log.info(f'Evaluation saved to: {self.eval_dir}')
        else:
            self.eval_dir = os.devnull
            self._log.info(f'Evaluation will not be saved.')
    #         self._aux_data_history = deque(maxlen=100)
    
        self.trained_epochs = 0
        self.trained_steps = 0
    
    
    @property
    def diffuser(self):
        return self._diffuser

    @property
    def model(self):
        return self._model

    @property
    def conf(self):
        return self._conf
    
    def create_dataset(self):
        
        
        
        #grab the first 3 atoms which are N,CA,C
        rr = np.load(self.paths_train)
        coords_train = [rr[f] for f in rr.files][0][:self.limit,:]
        
        rr = np.load(self.paths_valid)
        coords_valid = [rr[f] for f in rr.files][0][:self.limit,:]
        
        self.B
        
        device='cuda'
        
        prot_trainData = Data_Graph.ProteinBB_Dataset(coords_train[:self.limit], n_nodes=self.L,
                      n_atoms=5, coord_div=10, cast_type=torch.float32)
        train_dL = DataLoader(prot_trainData, batch_size=self.B, shuffle=True, drop_last=True)
        
        prot_validData = Data_Graph.ProteinBB_Dataset(coords_valid[:self.limit], n_nodes=self.L,
                      n_atoms=5, coord_div=10, cast_type=torch.float32)
        valid_dL = DataLoader(prot_trainData, batch_size=self.B, shuffle=True, drop_last=True)
        
        return train_dL, valid_dL
    
    def start_training(self, return_logs=False):


        self._model = self._model.to(self.device)
        print(f"Using device: {self.device}")

        self._model.train()
        (train_loader, valid_loader) = self.create_dataset()

        logs = []
        print('number of epochs', self.num_epoch)
        for epoch in range(self.trained_epochs, self.num_epoch):
            self.trained_epochs = epoch
            print('epoch', epoch)
            epoch_log = self.train_epoch(
                train_loader,
                valid_loader,
                return_logs=return_logs
            )
            if return_logs:
                logs.append(epoch_log)

        self._log.info('Done')
        return logs
    
    
    
    def train_epoch(self, train_loader, valid_loader, return_logs=False):
        
        log_lossses = defaultdict(list)
        
        global_logs = []
        
        log_time = time.time()
        step_time = time.time()
        losskeeper = []
        for train_feats in train_loader:
            
            #train_feats = tree.map_structure(lambda x: x.to(device), train_feats)
            loss, aux_data = self.update_fn(train_feats)
            losskeeper.append(float(loss.detach().cpu()))
            
            if return_logs:
                global_logs.append(loss)
            
            for k,v in aux_data.items():
                log_lossses[k].append(du.move_to_np(v))
            
            self.trained_steps += 1
            
            # Logging to terminal
            if self.trained_steps == 1 or self.trained_steps % self.log_freq == 0:
                elapsed_time = time.time() - log_time
                log_time = time.time()
                step_per_sec = self.log_freq / elapsed_time
                
                rolling_losses = tree.map_structure(np.mean, log_lossses)
                loss_log = ' '.join([
                    f'{k}={v[0]:.4f}'
                    for k,v in rolling_losses.items() if 'batch' not in k
                ])
                
                self._log.info(
                    f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                log_lossses = defaultdict(list)
                
                print(f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                print(np.mean(losskeeper[-1000:]))

            # Take checkpoint
            
            if self.ckpt_dir is not None and (
                    (self.trained_steps % self.ckpt_freq) == 0
                    or (self.early_ckpt and self.trained_steps == 2)
                ):
                ckpt_path = os.path.join(
                    self.ckpt_dir, f'step_{self.trained_steps}.pth')
                du.write_checkpoint(
                    ckpt_path,
                    copy.deepcopy(self.model.state_dict()),
                    self._conf,
                    copy.deepcopy(self._optimizer.state_dict()),
                    self.trained_epochs,
                    self.trained_steps,
                    logger=self._log,
                    use_torch=True
                )
                

                # Run evaluation
                self._log.info(f'Running evaluation of {ckpt_path}')
                start_time = time.time()
                self.eval_dir = 'output/'
                eval_dir = os.path.join(
                    self.eval_dir, f'step_{self.trained_steps}')
                print(eval_dir)
                os.makedirs(eval_dir, exist_ok=True)
                #_
                #figure out endpoints for this
#                 ckpt_metrics = self.eval_fn(
#                     eval_dir, valid_loader, device,
#                     noise_scale=self.noise_scale
#                 )
                eval_time = time.time() - start_time
                self._log.info(f'Finished evaluation in {eval_time:.2f}s')
            else:
                ckpt_metrics = None
                eval_time = None


            if torch.isnan(loss):                
                raise Exception(f'NaN encountered')

        if return_logs:
            return global_logs


    def update_fn(self, data):
        """Updates the state using some data and returns metrics."""
        self._optimizer.zero_grad()
        loss, aux_data = self.loss_fn(data)
        loss.backward()
        self._optimizer.step()
        return loss, aux_data
    
    def loss_fn(self, bb_dict, t_val=None):
        
        def convert_pV_to_points(dict_in, L, key_in='bb_firstp', return_indiv=False):
            """Concatenates to xyz from Calpha+atom vectors"""
            CA_fp  = dict_in[key_in]['CA'].reshape(self.B, L, 3).to(self.device)
            NC_fp = CA_fp + dict_in[key_in]['N_CA'].reshape(self.B, L, 3).to(self.device)
            CC_fp = CA_fp + dict_in[key_in]['C_CA'].reshape(self.B, L, 3).to(self.device)
            fp =  torch.cat((NC_fp,CA_fp,CC_fp),dim=2).reshape(self.B,L,3,3)
            if return_indiv:
                return fp, CA_fp, NC_fp, CC_fp
            return fp
        
        if t_val is not None:
            noised_dict = self.fnd.forward(bb_dict, t_vec=t_val)
        else:
            #generates with random t
            noised_dict = self.fnd.forward(bb_dict)
        
        batched_t = noised_dict['t_vec'].to(self.device)
        
        #not converted to distance by multiplying by bond distance, seems to work better
        true =  convert_pV_to_points(noised_dict, self.L, key_in='bb_shifted')
        noise_xyz, CA_n, NC_n, CC_n =   convert_pV_to_points(noised_dict, self.L, key_in='bb_noised', return_indiv=True)
                                                                                                            
        #prepare graphs
        feat_dict = self.mkg.prep_for_network(noised_dict, cuda=True)
        out  = self._model(feat_dict,batched_t)
                                                                    
        #FAPE Loss for the prediction
        CA_p = out['1'][:,0,:].reshape(self.B, self.L, 3) + CA_n #translation of Calpha
        Qs = out['1'][:,1,:] # rotation , convert from x,y,z (Quat) to rotate input vectors
        Qs = Qs.unsqueeze(1).repeat((1,2,1))
        Qs = torch.cat((torch.ones((self.B*self.L,2,1), device=Qs.device),Qs),dim=-1).reshape(self.B, self.L, 2, 4)
        Qs = normQ(Qs)
        Rs = Qs2Rs(Qs)
        N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(self.B, self.L, 3).to(self.device),
                                noised_dict['bb_noised']['C_CA'].reshape(self.B, self.L, 3).to(self.device)),dim=2).reshape(self.B,self.L,2,1,3)
        rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
        NC_p = CA_p + rot_vecs[:,:,0,:]*self.N_CA_dist #comparable but seems better not have it for true, but have it for pred
        CC_p = CA_p + rot_vecs[:,:,1,:]*self.C_CA_dist #maybe this helep prevent 

        pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(self.B,self.L,3,3)
                                                                
                                                                                                                     
        fp = convert_pV_to_points(noised_dict, 1, key_in='bb_firstp')
        lp = convert_pV_to_points(noised_dict,1,  key_in='bb_lastp')                                             
        real_mask = noised_dict['real_nodes_mask'].to(self.device)
        score_scales = noised_dict['score_scales'].to(self.device)

        lr, lr_d = FAPE_loss_real(pred, true, score_scales, real_mask,  d_clamp=10.0, d_clamp_inter=30.0,
                       A=10.0, gamma=1.0, eps=1e-6)

        ln, ln_d = FAPE_loss_null(pred, fp, lp, real_mask, score_scales,  d_clamp=10.0,
                           d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
        
        ln = ln*self.score_weights['3D_null']
        lr = lr*self.score_weights['3D_real']

        structure_loss = lr + ln

        #score for node feats determining whether node is real or fake
        nf_pred = out['0']

        nf_feat_dim = noised_dict['real_nodes_noise'].shape[-1]
        nf_true = torch.ones(noised_dict['real_nodes_mask'].shape+(nf_feat_dim,) + (1,),
                             dtype=torch.float,device = self.device)

        nf_real_mask_mult = real_mask.unsqueeze(-1).unsqueeze(-1).to(self.device)
        nf_true = nf_true*nf_real_mask_mult

        nf_pred = nf_pred.reshape(self.B,-1,nf_feat_dim)
        pred_nf_loss = torch.sum(torch.abs(nf_true.squeeze()-nf_pred),dim=-1) #absolute value loss

        ss_scales = to_cuda(noised_dict['score_scales'])[:,None,None]
        pnfloss = (torch.sum((pred_nf_loss*ss_scales/self.L)))*self.score_weights['nf_real']

        final_loss = structure_loss + pnfloss
        
        aux_loss = {'pnf_loss':pnfloss,
                    'structure_loss':structure_loss,
                    'structure_null':ln,
                    'structure_real':lr}
                                                                
        return final_loss, aux_loss
                                                                
                                                                
            
                                                                
            

In [ ]:
#need to add eval code, / dump conversion

In [141]:
exp = Experiment()

In [142]:
tl, vl = exp.create_dataset()

In [143]:
exp.start_training()

Using device: cuda
number of epochs 100
epoch 0
[1]: pnf_loss=75.6255 structure_loss=3.5394 structure_null=1.3346 structure_real=2.2048, steps/sec=44.85314
79.16490936279297
output/step_2
[50]: pnf_loss=36.9373 structure_loss=3.7276 structure_null=1.4043 structure_real=2.3233, steps/sec=1.06283
77.387943649292
[100]: pnf_loss=49.8326 structure_loss=3.6549 structure_null=1.1571 structure_real=2.4978, steps/sec=1.07475
73.74939380645752


KeyboardInterrupt: 

In [ ]:
import tree


In [ ]:
def train_epoch(self, train_loader, valid_loader, device, return_logs=False):
        log_lossses = defaultdict(list)
        global_logs = []
        log_time = time.time()
        step_time = time.time()
        losskeeper = []
        for train_feats in train_loader:
            train_feats = tree.map_structure(lambda x: x.to(device), train_feats)
            loss, aux_data = self.update_fn(train_feats)
            losskeeper.append(float(loss.detach().cpu()))
            if return_logs:
                global_logs.append(loss)
            for k,v in aux_data.items():
                log_lossses[k].append(du.move_to_np(v))
            self.trained_steps += 1

            # Logging to terminal
            if self.trained_steps == 1 or self.trained_steps % self._exp_conf.log_freq == 0:
                elapsed_time = time.time() - log_time
                log_time = time.time()
                step_per_sec = self._exp_conf.log_freq / elapsed_time
                rolling_losses = tree.map_structure(np.mean, log_lossses)
                loss_log = ' '.join([
                    f'{k}={v[0]:.4f}'
                    for k,v in rolling_losses.items() if 'batch' not in k
                ])
                self._log.info(
                    f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                log_lossses = defaultdict(list)
                
                print(f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                print(np.mean(losskeeper[-1000:]))

            # Take checkpoint
            if self._exp_conf.ckpt_dir is not None and (
                    (self.trained_steps % self._exp_conf.ckpt_freq) == 0
                    or (self._exp_conf.early_ckpt and self.trained_steps == 2)
                ):
                ckpt_path = os.path.join(
                    self._exp_conf.ckpt_dir, f'step_{self.trained_steps}.pth')
                du.write_checkpoint(
                    ckpt_path,
                    copy.deepcopy(self.model.state_dict()),
                    self._conf,
                    copy.deepcopy(self._optimizer.state_dict()),
                    self.trained_epochs,
                    self.trained_steps,
                    logger=self._log,
                    use_torch=True
                )
                

                # Run evaluation
                self._log.info(f'Running evaluation of {ckpt_path}')
                start_time = time.time()
                self._exp_conf.eval_dir = 'output/'
                eval_dir = os.path.join(
                    self._exp_conf.eval_dir, f'step_{self.trained_steps}')
                print(eval_dir)
                os.makedirs(eval_dir, exist_ok=True)
                ckpt_metrics = self.eval_fn(
                    eval_dir, valid_loader, device,
                    noise_scale=self._exp_conf.noise_scale
                )
                eval_time = time.time() - start_time
                self._log.info(f'Finished evaluation in {eval_time:.2f}s')
            else:
                ckpt_metrics = None
                eval_time = None


            if torch.isnan(loss):
                if self._use_wandb:
                    wandb.alert(
                        title="Encountered NaN loss",
                        text=f"Loss NaN after {self.trained_epochs} epochs, {self.trained_steps} steps"
                    )
                raise Exception(f'NaN encountered')

        if return_logs:
            return global_logs

In [ ]:


def model_step_null(noised_dict, batched_t, graph_maker, graph_unet, train=True):
    #prep coordinates for output display from and comparison via FAPE
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to(device)
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to(device)#not mult by bond distance, seems to help?
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to(device)#not mult 
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to(device)
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device)
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

    #prepare graphs
    feat_dict = graph_maker.prep_for_network(noised_dict, cuda=True)
    out =graph_unet(feat_dict,batched_t)
    
    
    #FAPE Loss for the prediction
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation , convert from x,y,z (Quat) to rotate input vectors
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device),
                            noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)),dim=2).reshape(B,L,2,1,3)
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #comparable but seems better not have it for true, but have it for pred
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #maybe this helep prevent 

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    #divide loss by real and null nodes
    
    fp, lp  = convert_pV_to_points(noised_dict)

    real_mask = noised_dict['real_nodes_mask'].to('cuda')
    score_scales = noised_dict['score_scales'].to('cuda')

    lr, lr_d = FAPE_loss_real(pred, true, score_scales, real_mask,  d_clamp=10.0, d_clamp_inter=30.0,
                   A=10.0, gamma=1.0, eps=1e-6)
    
    ln, ln_d = FAPE_loss_null(pred, fp, lp, real_mask, score_scales,  d_clamp=10.0,
                       d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
    
    structure_loss = lr*score_weights['3D_real']+ln*score_weights['3D_null']
    
    #score for node feats determining whether node is real or fake
    nf_pred = out['0']

    nf_feat_dim = noised_dict['real_nodes_noise'].shape[-1]
    nf_true = torch.ones(noised_dict['real_nodes_mask'].shape+(nf_feat_dim,) + (1,),
                         dtype=torch.float,device=device)

    nf_real_mask_mult = real_mask.unsqueeze(-1).unsqueeze(-1).to(device)
    nf_true = nf_true*nf_real_mask_mult

    nf_pred = nf_pred.reshape(B,-1,nf_feat_dim)
    pred_nf_loss = torch.sum(torch.abs(nf_true.squeeze()-nf_pred),dim=-1) #absolute value loss
    pred_nf_loss = pred_nf_loss.to(device)
    
    ss_scales = to_cuda(noised_dict['score_scales'])[:,None,None]
    pnfloss = (torch.sum((pred_nf_loss*ss_scales/L)))*score_weights['nf_real']
    
    final_loss = structure_loss + pnfloss
    
    
    return final_loss, pnfloss.detach().cpu(), structure_loss.detach().cpu(), lr.detach().cpu(), ln.detach().cpu()

In [ ]:
def train_epoch(self, train_loader, valid_loader, device, return_logs=False):
        log_lossses = defaultdict(list)
        global_logs = []
        log_time = time.time()
        step_time = time.time()
        losskeeper = []
        for train_feats in train_loader:
            train_feats = tree.map_structure(lambda x: x.to(device), train_feats)
            loss, aux_data = self.update_fn(train_feats)
            losskeeper.append(float(loss.detach().cpu()))
            if return_logs:
                global_logs.append(loss)
            for k,v in aux_data.items():
                log_lossses[k].append(du.move_to_np(v))
            self.trained_steps += 1

            # Logging to terminal
            if self.trained_steps == 1 or self.trained_steps % self._exp_conf.log_freq == 0:
                elapsed_time = time.time() - log_time
                log_time = time.time()
                step_per_sec = self._exp_conf.log_freq / elapsed_time
                rolling_losses = tree.map_structure(np.mean, log_lossses)
                loss_log = ' '.join([
                    f'{k}={v[0]:.4f}'
                    for k,v in rolling_losses.items() if 'batch' not in k
                ])
                self._log.info(
                    f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                log_lossses = defaultdict(list)
                
                print(f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                print(np.mean(losskeeper[-1000:]))

            # Take checkpoint
            if self._exp_conf.ckpt_dir is not None and (
                    (self.trained_steps % self._exp_conf.ckpt_freq) == 0
                    or (self._exp_conf.early_ckpt and self.trained_steps == 2)
                ):
                ckpt_path = os.path.join(
                    self._exp_conf.ckpt_dir, f'step_{self.trained_steps}.pth')
                du.write_checkpoint(
                    ckpt_path,
                    copy.deepcopy(self.model.state_dict()),
                    self._conf,
                    copy.deepcopy(self._optimizer.state_dict()),
                    self.trained_epochs,
                    self.trained_steps,
                    logger=self._log,
                    use_torch=True
                )
                

                # Run evaluation
                self._log.info(f'Running evaluation of {ckpt_path}')
                start_time = time.time()
                self._exp_conf.eval_dir = 'output/'
                eval_dir = os.path.join(
                    self._exp_conf.eval_dir, f'step_{self.trained_steps}')
                print(eval_dir)
                os.makedirs(eval_dir, exist_ok=True)
                ckpt_metrics = self.eval_fn(
                    eval_dir, valid_loader, device,
                    noise_scale=self._exp_conf.noise_scale
                )
                eval_time = time.time() - start_time
                self._log.info(f'Finished evaluation in {eval_time:.2f}s')
            else:
                ckpt_metrics = None
                eval_time = None


            if torch.isnan(loss):
                if self._use_wandb:
                    wandb.alert(
                        title="Encountered NaN loss",
                        text=f"Loss NaN after {self.trained_epochs} epochs, {self.trained_steps} steps"
                    )
                raise Exception(f'NaN encountered')

        if return_logs:
            return global_logs

In [ ]:


    
# class EMA(nn.Module):
#     def __init__(self, mu):
#         super(EMA, self).__init__()
#         self.mu = mu
#         self.shadow = {}

#     def register(self, name, val):
#         self.shadow[name] = val.clone()

#     def forward(self, name, x):
#         assert name in self.shadow
#         new_average = (1.0 - self.mu) * x + self.mu * self.shadow[name]
#         self.shadow[name] = new_average.clone()
#         return new_average

class Experiment:

    def __init__(self,*,config_path= 'config/base_static_4H_norot.yaml', simple=True, limit=5028):
        """Initialize experiment.
        Currently does not incorporate psi pred (just taken from starting points)
        Args:
            exp_cfg: Experiment configuration.
        """
        with open(config_path, 'r') as file:
            config = yaml.safe_load(file)
        conf = Struct(config)
        
        logging.basicConfig(filename='test.log', level=logging.INFO)
        self._log = logging.getLogger(__name__)
        
        self.simple = simple
        self.limit = limit
        
        self._use_wandb = False
        
        self.rot=False
        self.bbloss=False
        self.device='cuda'
        
        # Configs
        self._conf = conf
        self._exp_conf = conf.experiment
        self._diff_conf = conf.diffuser
        self._model_conf = conf.model
        self._data_conf = conf.data
        self.ipa_conf = conf.model.ipa
        
        
        self.B=self._exp_conf.batch_size
        L=65 #HARDDDCODE
        
        self._diffuser = se3_diffuser.SE3Diffuser(self._diff_conf)
        self._model = ScoreNetwork(
            self._model_conf, self._diffuser,B=self.B,L=L)
        num_parameters = sum(p.numel() for p in self._model.parameters())
        self._exp_conf.num_parameters = num_parameters
        self._log.info(f'Number of model parameters {num_parameters}')
#         self._optimizer = EMA(0.980)
#         for name, param in self._model.named_parameters():
#             if param.requires_grad:
#                 self._optimizer.register(name, param.data)
        
        
        self._optimizer = torch.optim.Adam(
            self._model.score_model.parameters(), lr=self._exp_conf.learning_rate)
        dt_string = datetime.now().strftime("%dD_%mM_%YY_%Hh_%Mm_%Ss")
        self._exp_conf.ckpt_dir = 'log/'
        if self._exp_conf.ckpt_dir is not None:
            # Set-up checkpoint location
            ckpt_dir = os.path.join(
                self._exp_conf.ckpt_dir,
                self._exp_conf.name,
                dt_string)
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir, exist_ok=True)
            self._exp_conf.ckpt_dir = ckpt_dir
            self._log.info(f'Checkpoints saved to: {ckpt_dir}')
        else:  
            self._log.info('Checkpoint not being saved.')
            
        if self._exp_conf.eval_dir is not None :
            eval_dir = os.path.join(
                self._exp_conf.eval_dir,
                self._exp_conf.name,
                dt_string)
            self._exp_conf.eval_dir = eval_dir
            self._log.info(f'Evaluation saved to: {eval_dir}')
        else:
            self._exp_conf.eval_dir = os.devnull
            self._log.info(f'Evaluation will not be saved.')
    #         self._aux_data_history = deque(maxlen=100)

        # Warm starting
        ckpt_model = None
        ckpt_opt = None
        self.trained_epochs = 0
        self.trained_steps = 0

        # Initialize experiment objects

        if ckpt_model is not None:
            ckpt_model = {k.replace('module.', ''):v for k,v in ckpt_model.items()}
            self._model.load_state_dict(ckpt_model, strict=True)

        num_parameters = sum(p.numel() for p in self._model.parameters())
        self._exp_conf.num_parameters = num_parameters
        self._log.info(f'Number of model parameters {num_parameters}')
        self._optimizer = torch.optim.Adam(
            self._model.parameters(), lr=self._exp_conf.learning_rate)
        if ckpt_opt is not None:
            self._optimizer.load_state_dict(ckpt_opt)

        dt_string = datetime.now().strftime("%dD_%mM_%YY_%Hh_%Mm_%Ss")
        if self._exp_conf.ckpt_dir is not None:
            # Set-up checkpoint location
            ckpt_dir = os.path.join(
                self._exp_conf.ckpt_dir,
                self._exp_conf.name,
                dt_string)
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir, exist_ok=True)
            self._exp_conf.ckpt_dir = ckpt_dir
            self._log.info(f'Checkpoints saved to: {ckpt_dir}')
        else:  
            self._log.info('Checkpoint not being saved.')
        if self._exp_conf.eval_dir is not None :
            eval_dir = os.path.join(
                self._exp_conf.eval_dir,
                self._exp_conf.name,
                dt_string)
            self._exp_conf.eval_dir = eval_dir
            self._log.info(f'Evaluation saved to: {eval_dir}')
        else:
            self._exp_conf.eval_dir = os.devnull
            self._log.info(f'Evaluation will not be saved.')
        self._aux_data_history = deque(maxlen=100)
        
    @property
    def diffuser(self):
        return self._diffuser

    @property
    def model(self):
        return self._model

    @property
    def conf(self):
        return self._conf
    
    def create_dataset(self):
        self._exp_conf.eval_batch_size=self.B
        #valid limit set
        num_workers = self._exp_conf.num_loader_workers
        train_dataset = pdb_data_loader.PdbDataset(
                    data_conf=self._data_conf,
                    diffuser=self._diffuser,
                    is_training=True,
                    simple = self.simple
                )
        valid_dataset = pdb_data_loader.PdbDataset(
                    data_conf=self._data_conf,
                    diffuser=self._diffuser,
                    is_training=False,
                    simple = self.simple
                )

        train_dataset.csv = train_dataset.csv.iloc[:self.limit]
        valid_dataset.csv = valid_dataset.csv.iloc[:256] #valid

        valid_sampler=None
        train_sampler = pdb_data_loader.TrainSampler(
                data_conf=self._data_conf,
                dataset=train_dataset,
                batch_size=self.B,
                sample_mode=self._exp_conf.sample_mode,
            )


        train_loader = du.create_data_loader(
            train_dataset,
            sampler=train_sampler,
            np_collate=False,
            length_batch=False,
            batch_size=self.B,
            shuffle=False,
            num_workers=num_workers,
            drop_last=False,
            max_squared_res=100*2,
        )
        
        # Loaders
        
        train_loader = du.create_data_loader(
            train_dataset,
            sampler=train_sampler,
            np_collate=False,
            length_batch=False,
            batch_size=self.B,
            shuffle=False,
            num_workers=num_workers,
            drop_last=False,
            max_squared_res=self._exp_conf.max_squared_res,
        )
        
        #####TRAIN DATASET again##########
        valid_loader = du.create_data_loader(
            valid_dataset,
            sampler=valid_sampler,
            np_collate=False,
            length_batch=False,
            batch_size=self.B,
            shuffle=False,
            num_workers=1,
            drop_last=False,
        )
        
        
        
        return train_loader, train_sampler, valid_loader, valid_sampler
    
    def start_training(self, return_logs=False):

        # GPU mode
        if torch.cuda.is_available():
            device = f"cuda"
            self._model = self.model.to(device)
            print(f"Using device: {device}")
            self._log.info(f"Using device: {device}")
        else:
            device = 'cpu'
            self._model = self.model.to(device)
            self._log.info(f"Using device: {device}")

        self._model.train()
        #removed valid
        (train_loader, train_sampler, valid_loader, valid_sampler) = self.create_dataset()

        logs = []
        print(self._exp_conf.num_epoch)
        for epoch in range(self.trained_epochs, self._exp_conf.num_epoch):
            if train_sampler is not None:
                train_sampler.set_epoch(epoch)
            self.trained_epochs = epoch
            print(epoch)
            epoch_log = self.train_epoch(
                train_loader,
                valid_loader,
                device,
                return_logs=return_logs
            )
            if return_logs:
                logs.append(epoch_log)

        self._log.info('Done')
        return logs
    
    def train_epoch(self, train_loader, valid_loader, device, return_logs=False):
        log_lossses = defaultdict(list)
        global_logs = []
        log_time = time.time()
        step_time = time.time()
        losskeeper = []
        for train_feats in train_loader:
            train_feats = tree.map_structure(lambda x: x.to(device), train_feats)
            loss, aux_data = self.update_fn(train_feats)
            losskeeper.append(float(loss.detach().cpu()))
            if return_logs:
                global_logs.append(loss)
            for k,v in aux_data.items():
                log_lossses[k].append(du.move_to_np(v))
            self.trained_steps += 1

            # Logging to terminal
            if self.trained_steps == 1 or self.trained_steps % self._exp_conf.log_freq == 0:
                elapsed_time = time.time() - log_time
                log_time = time.time()
                step_per_sec = self._exp_conf.log_freq / elapsed_time
                rolling_losses = tree.map_structure(np.mean, log_lossses)
                loss_log = ' '.join([
                    f'{k}={v[0]:.4f}'
                    for k,v in rolling_losses.items() if 'batch' not in k
                ])
                self._log.info(
                    f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                log_lossses = defaultdict(list)
                
                print(f'[{self.trained_steps}]: {loss_log}, steps/sec={step_per_sec:.5f}')
                print(np.mean(losskeeper[-1000:]))

            # Take checkpoint
            if self._exp_conf.ckpt_dir is not None and (
                    (self.trained_steps % self._exp_conf.ckpt_freq) == 0
                    or (self._exp_conf.early_ckpt and self.trained_steps == 2)
                ):
                ckpt_path = os.path.join(
                    self._exp_conf.ckpt_dir, f'step_{self.trained_steps}.pth')
                du.write_checkpoint(
                    ckpt_path,
                    copy.deepcopy(self.model.state_dict()),
                    self._conf,
                    copy.deepcopy(self._optimizer.state_dict()),
                    self.trained_epochs,
                    self.trained_steps,
                    logger=self._log,
                    use_torch=True
                )
                

                # Run evaluation
                self._log.info(f'Running evaluation of {ckpt_path}')
                start_time = time.time()
                self._exp_conf.eval_dir = 'output/'
                eval_dir = os.path.join(
                    self._exp_conf.eval_dir, f'step_{self.trained_steps}')
                print(eval_dir)
                os.makedirs(eval_dir, exist_ok=True)
                ckpt_metrics = self.eval_fn(
                    eval_dir, valid_loader, device,
                    noise_scale=self._exp_conf.noise_scale
                )
                eval_time = time.time() - start_time
                self._log.info(f'Finished evaluation in {eval_time:.2f}s')
            else:
                ckpt_metrics = None
                eval_time = None


            if torch.isnan(loss):
                if self._use_wandb:
                    wandb.alert(
                        title="Encountered NaN loss",
                        text=f"Loss NaN after {self.trained_epochs} epochs, {self.trained_steps} steps"
                    )
                raise Exception(f'NaN encountered')

        if return_logs:
            return global_logs
        
    def eval_fn(self, eval_dir, valid_loader, device, min_t=None, num_t=None, noise_scale=1.0):
        ckpt_eval_metrics = []
        self._exp_conf.eval_batch_size=self.B
        for valid_feats, pdb_names in valid_loader:
            res_mask = du.move_to_np(valid_feats['res_mask'].bool())
            fixed_mask = du.move_to_np(valid_feats['fixed_mask'].bool())
            aatype = du.move_to_np(valid_feats['aatype'])
            gt_prot = du.move_to_np(valid_feats['atom37_pos'])
            batch_size = res_mask.shape[0]
            valid_feats = tree.map_structure(
                lambda x: x.to(device), valid_feats)
            
            # Run inference
            infer_out = self.inference_fn(
                valid_feats, min_t=min_t, num_t=num_t, noise_scale=noise_scale)
            final_prot = infer_out['prot_traj'][0]
            for i in range(batch_size):
                num_res = int(np.sum(res_mask[i]).item())
                unpad_fixed_mask = fixed_mask[i][res_mask[i]]
                unpad_diffused_mask = 1 - unpad_fixed_mask
                unpad_prot = final_prot[i][res_mask[i]]
                unpad_gt_prot = gt_prot[i][res_mask[i]]
                unpad_gt_aatype = aatype[i][res_mask[i]]
                percent_diffused = np.sum(unpad_diffused_mask) / num_res

                # Extract argmax predicted aatype
                saved_path = au.write_prot_to_pdb(
                    unpad_prot,
                    os.path.join(
                        eval_dir,
                        f'len_{num_res}_sample_{i}_diffused_{percent_diffused:.2f}.pdb'
                    ),
                    no_indexing=True,
                    b_factors=np.tile(1 - unpad_fixed_mask[..., None], 37) * 100
                )
                try:
                    sample_metrics = metrics.protein_metrics(
                        pdb_path=saved_path,
                        atom37_pos=unpad_prot,
                        gt_atom37_pos=unpad_gt_prot,
                        gt_aatype=unpad_gt_aatype,
                        diffuse_mask=unpad_diffused_mask,
                         )
                except ValueError as e:
                    self._log.warning(
                        f'Failed evaluation of length {num_res} sample {i}: {e}')
                    continue
                sample_metrics['step'] = self.trained_steps
                sample_metrics['num_res'] = num_res
                sample_metrics['fixed_residues'] = np.sum(unpad_fixed_mask)
                sample_metrics['diffused_percentage'] = percent_diffused
                sample_metrics['sample_path'] = saved_path
                sample_metrics['gt_pdb'] = pdb_names[i]
                ckpt_eval_metrics.append(sample_metrics)

        # Save metrics as CSV.
        eval_metrics_csv_path = os.path.join(eval_dir, 'metrics.csv')
        ckpt_eval_metrics = pd.DataFrame(ckpt_eval_metrics)
        ckpt_eval_metrics.to_csv(eval_metrics_csv_path, index=False)
        return ckpt_eval_metrics
    
    
    def _self_conditioning(self, batch):
        model_sc = self.model(batch)
        batch['sc_ca_t'] = model_sc['rigids'][..., 4:]
        return batch
    
    def loss_fn(self,batch):
        
        
        
        model_out = self._model(batch) #score network calls graph maker/gu_net
    
        #loss
        bb_mask = batch['res_mask']
        diffuse_mask = 1 - batch['fixed_mask']
        loss_mask = bb_mask * diffuse_mask
        batch_size, num_res = bb_mask.shape

        gt_rot_score = batch['rot_score']
        gt_trans_score = batch['trans_score']
        rot_score_scaling = batch['rot_score_scaling']
        trans_score_scaling = batch['trans_score_scaling']
        batch_loss_mask = torch.any(bb_mask, dim=-1)

        pred_rot_score = model_out['rot_score'] * diffuse_mask[..., None]
        pred_trans_score = model_out['trans_score'] * diffuse_mask[..., None]

        # Translation score loss
        trans_score_mse = (gt_trans_score - pred_trans_score)**2 * loss_mask[..., None]
        trans_score_loss = torch.sum(
            trans_score_mse / trans_score_scaling[:, None, None]**2,
            dim=(-1, -2)
        ) / (loss_mask.sum(dim=-1) + 1e-10)

        # Translation x0 loss
        gt_trans_x0 = batch['rigids_0'][..., 4:] * self._exp_conf.coordinate_scaling
        pred_trans_x0 = model_out['rigids'][..., 4:] * self._exp_conf.coordinate_scaling
        trans_x0_loss = torch.sum(
            (gt_trans_x0 - pred_trans_x0)**2 * loss_mask[..., None],
            dim=(-1, -2)
        ) / (loss_mask.sum(dim=-1) + 1e-10)

        trans_loss = (
            trans_score_loss * (batch['t'] > self._exp_conf.trans_x0_threshold)
            + trans_x0_loss * (batch['t'] <= self._exp_conf.trans_x0_threshold)
        )
        if self.rot:
            # Rotation loss
            if self._exp_conf.separate_rot_loss:
                gt_rot_angle = torch.norm(gt_rot_score, dim=-1, keepdim=True)
                gt_rot_axis = gt_rot_score / (gt_rot_angle + 1e-6)

                pred_rot_angle = torch.norm(pred_rot_score, dim=-1, keepdim=True)
                pred_rot_axis = pred_rot_score / (pred_rot_angle + 1e-6)

                # Separate loss on the axis
                axis_loss = (gt_rot_axis - pred_rot_axis)**2 * loss_mask[..., None]
                axis_loss = torch.sum(
                    axis_loss, dim=(-1, -2)
                ) / (loss_mask.sum(dim=-1) + 1e-10)

                # Separate loss on the angle
                angle_loss = (gt_rot_angle - pred_rot_angle)**2 * loss_mask[..., None]
                angle_loss = torch.sum(
                    angle_loss / rot_score_scaling[:, None, None]**2,
                    dim=(-1, -2)
                ) / (loss_mask.sum(dim=-1) + 1e-10)
                angle_loss *= self._exp_conf.rot_loss_weight
                angle_loss *= batch['t'] > self._exp_conf.rot_loss_t_threshold
                rot_loss = angle_loss + axis_loss
            else:
                rot_mse = (gt_rot_score - pred_rot_score)**2 * loss_mask[..., None]
                rot_loss = torch.sum(
                    rot_mse / rot_score_scaling[:, None, None]**2,
                    dim=(-1, -2)
                ) / (loss_mask.sum(dim=-1) + 1e-10)
                rot_loss *= self._exp_conf.rot_loss_weight
                rot_loss *= batch['t'] > self._exp_conf.rot_loss_t_threshold
            rot_loss *= int(self._diff_conf.diffuse_rot)
            
        else:
            rot_loss = torch.zeros_like(trans_loss,device=self.device)
        

        if self.bbloss:
            # Backbone atom loss
            pred_atom37 = model_out['atom37'][:, :, :5]
            gt_rigids = ru.Rigid.from_tensor_7(batch['rigids_0'].type(torch.float32))
            gt_psi = batch['torsion_angles_sin_cos'][..., 2, :]
            gt_atom37, atom37_mask, _, _ = all_atom.compute_backbone(
                gt_rigids, gt_psi)
            gt_atom37 = gt_atom37[:, :, :5]
            atom37_mask = atom37_mask[:, :, :5]

            gt_atom37 = gt_atom37.to(pred_atom37.device)
            atom37_mask = atom37_mask.to(pred_atom37.device)
            bb_atom_loss_mask = atom37_mask * loss_mask[..., None]
            bb_atom_loss = torch.sum(
                (pred_atom37 - gt_atom37)**2 * bb_atom_loss_mask[..., None],
                dim=(-1, -2, -3)
            ) / (bb_atom_loss_mask.sum(dim=(-1, -2)) + 1e-10)
            bb_atom_loss *= self._exp_conf.bb_atom_loss_weight
            bb_atom_loss *= batch['t'] < self._exp_conf.bb_atom_loss_t_filter
            bb_atom_loss *= self._exp_conf.aux_loss_weight


            # Pairwise distance loss
            gt_flat_atoms = gt_atom37.reshape([batch_size, num_res*5, 3])
            gt_pair_dists = torch.linalg.norm(
                gt_flat_atoms[:, :, None, :] - gt_flat_atoms[:, None, :, :], dim=-1)
            pred_flat_atoms = pred_atom37.reshape([batch_size, num_res*5, 3])
            pred_pair_dists = torch.linalg.norm(
                pred_flat_atoms[:, :, None, :] - pred_flat_atoms[:, None, :, :], dim=-1)

            flat_loss_mask = torch.tile(loss_mask[:, :, None], (1, 1, 5))
            flat_loss_mask = flat_loss_mask.reshape([batch_size, num_res*5])
            flat_res_mask = torch.tile(bb_mask[:, :, None], (1, 1, 5))
            flat_res_mask = flat_res_mask.reshape([batch_size, num_res*5])

            gt_pair_dists = gt_pair_dists * flat_loss_mask[..., None]
            pred_pair_dists = pred_pair_dists * flat_loss_mask[..., None]
            pair_dist_mask = flat_loss_mask[..., None] * flat_res_mask[:, None, :]

            # No loss on anything >6A
            proximity_mask = gt_pair_dists < 6
            pair_dist_mask  = pair_dist_mask * proximity_mask

            dist_mat_loss = torch.sum(
                (gt_pair_dists - pred_pair_dists)**2 * pair_dist_mask,
                dim=(1, 2))
            dist_mat_loss /= (torch.sum(pair_dist_mask, dim=(1, 2)) - num_res)
            dist_mat_loss *= self._exp_conf.dist_mat_loss_weight
            dist_mat_loss *= batch['t'] < self._exp_conf.dist_mat_loss_t_filter
            dist_mat_loss *= self._exp_conf.aux_loss_weight
            
        else:
            bb_atom_loss = torch.zeros_like(trans_loss,device=self.device)
            dist_mat_loss = torch.zeros_like(trans_loss,device=self.device)

         
        final_loss = (
            rot_loss
            + trans_loss
            + bb_atom_loss
            + dist_mat_loss
        )

        def normalize_loss(x):
            return x.sum() /  (batch_loss_mask.sum() + 1e-9)

        aux_data = {
            'batch_train_loss': final_loss,
            'batch_rot_loss': rot_loss,
            'batch_trans_loss': trans_loss,
            'batch_bb_atom_loss': bb_atom_loss,
            'batch_dist_mat_loss': dist_mat_loss,
            'total_loss': normalize_loss(final_loss),
            'rot_loss': normalize_loss(rot_loss),
            'trans_loss': normalize_loss(trans_loss),
            'bb_atom_loss': normalize_loss(bb_atom_loss),
            'dist_mat_loss': normalize_loss(dist_mat_loss),
            'examples_per_step': torch.tensor(batch_size),
            'res_length': torch.mean(torch.sum(bb_mask, dim=-1)),
        }

        return normalize_loss(final_loss),aux_data
        
        
    def update_fn(self, data):
        """Updates the state using some data and returns metrics."""
        self._optimizer.zero_grad()
        loss, aux_data = self.loss_fn(data)
        loss.backward()
        self._optimizer.step()
        return loss, aux_data

    def _calc_trans_0(self, trans_score, trans_t, t):
        beta_t = self._diffuser._se3_diffuser._r3_diffuser.marginal_b_t(t)
        beta_t = beta_t[..., None, None]
        cond_var = 1 - torch.exp(-beta_t)
        return (trans_score * cond_var + trans_t) / torch.exp(-1/2*beta_t)

    def _set_t_feats(self, feats, t, t_placeholder):
        feats['t'] = t * t_placeholder
        rot_score_scaling, trans_score_scaling = self.diffuser.score_scaling(t)
        feats['rot_score_scaling'] = rot_score_scaling * t_placeholder
        feats['trans_score_scaling'] = trans_score_scaling * t_placeholder
        return feats

    def forward_traj(self, x_0, min_t, num_t):
        forward_steps = np.linspace(min_t, 1.0, num_t)[:-1]
        x_traj = [x_0]
        for t in forward_steps:
            x_t = self.diffuser.se3_diffuser._r3_diffuser.forward(
                x_traj[-1], t, num_t)
            x_traj.append(x_t)
        x_traj = torch.stack(x_traj, axis=0)
        return x_traj

    def inference_fn(
            self,
            data_init,
            num_t=None,
            min_t=None,
            center=True,
            aux_traj=False,
            self_condition=True,
            noise_scale=1.0,
        ):
        """Inference function.

        Args:
            data_init: Initial data values for sampling.
        """
        
        # Run reverse process.
        sample_feats = copy.deepcopy(data_init)
        device = sample_feats['rigids_t'].device
#         for k,v in sample_feats.items():
#             sample_feats[k] = torch.repeat_interleave(sample_feats[k],torch.tensor([2]*8).to(device),dim=0)
        device = sample_feats['rigids_t'].device
        if sample_feats['rigids_t'].ndim == 2:
            t_placeholder = torch.ones((1,)).to(device)#test remove
#             t_placeholder = torch.ones(
#                 (self.B,)).to(device)
        else:
            t_placeholder = torch.ones(
                (sample_feats['rigids_t'].shape[0],)).to(device)
        if num_t is None:
            num_t = self._data_conf.num_t
        if min_t is None:
            min_t = self._data_conf.min_t
        reverse_steps = np.linspace(min_t, 1.0, num_t)[::-1]
        dt = 1/num_t
        all_rigids = [du.move_to_np(copy.deepcopy(sample_feats['rigids_t']))]
        all_bb_prots = []
        all_trans_0_pred = []
        all_bb_0_pred = []
        with torch.no_grad():
#             if self._model_conf.embed.embed_self_conditioning and self_condition:
#                 sample_feats = self._set_t_feats(
#                     sample_feats, reverse_steps[0], t_placeholder)
#                 sample_feats = self._self_conditioning(sample_feats)
            for t in reverse_steps:
                if t > min_t:
                    sample_feats = self._set_t_feats(sample_feats, t, t_placeholder)
                    model_out = self._model(sample_feats)
                    #model_out = self.model(sample_feats)
                    rot_score = model_out['rot_score']
                    trans_score = model_out['trans_score']
                    rigid_pred = model_out['rigids']
#                     if self._model_conf.embed.embed_self_conditioning:
#                         sample_feats['sc_ca_t'] = rigid_pred[..., 4:]
                    fixed_mask = sample_feats['fixed_mask'] * sample_feats['res_mask']
                    diffuse_mask = (1 - sample_feats['fixed_mask']) * sample_feats['res_mask']
                    rigids_t = self.diffuser.reverse(
                        rigid_t=ru.Rigid.from_tensor_7(sample_feats['rigids_t']),
                        rot_score=du.move_to_np(rot_score),
                        trans_score=du.move_to_np(trans_score),
                        diffuse_mask=du.move_to_np(diffuse_mask),
                        t=t,
                        dt=dt,
                        center=center,
                        noise_scale=noise_scale,
                    )
                else:
                    model_out = self._model(sample_feats)
                    #model_out = self.model(sample_feats)
                    rigids_t = ru.Rigid.from_tensor_7(model_out['rigids'])
                sample_feats['rigids_t'] = rigids_t.to_tensor_7().to(device)
                if aux_traj:
                    all_rigids.append(du.move_to_np(rigids_t.to_tensor_7()))

                # Calculate x0 prediction derived from score predictions.
                gt_trans_0 = sample_feats['rigids_t'][..., 4:]
                pred_trans_0 = rigid_pred[..., 4:]
                trans_pred_0 = diffuse_mask[..., None] * pred_trans_0 + fixed_mask[..., None] * gt_trans_0
                psi_pred = model_out['psi']
                if aux_traj:
                    atom37_0 = all_atom.compute_backbone(
                        ru.Rigid.from_tensor_7(rigid_pred),
                        psi_pred
                    )[0]
                    all_bb_0_pred.append(du.move_to_np(atom37_0))
                    all_trans_0_pred.append(du.move_to_np(trans_pred_0))
                atom37_t = all_atom.compute_backbone(
                    rigids_t, psi_pred)[0]
                all_bb_prots.append(du.move_to_np(atom37_t))

        # Flip trajectory so that it starts from t=0.
        # This helps visualization.
        flip = lambda x: np.flip(np.stack(x), (0,))
        all_bb_prots = flip(all_bb_prots)
        if aux_traj:
            all_rigids = flip(all_rigids)
            all_trans_0_pred = flip(all_trans_0_pred)
            all_bb_0_pred = flip(all_bb_0_pred)

        ret = {
            'prot_traj': all_bb_prots,
        }
        if aux_traj:
            ret['rigid_traj'] = all_rigids
            ret['trans_traj'] = all_trans_0_pred
            ret['psi_pred'] = psi_pred[None]
            ret['rigid_0_traj'] = all_bb_0_pred
        return ret
        

In [2]:
import se3_diffuse.utils as du

In [3]:
from data_rigid_diffuser import so3_diffuser
from data_rigid_diffuser import r3_diffuser
from data_rigid_diffuser import oneHot_diffuser
from scipy.spatial.transform import Rotation
from data_rigid_diffuser import rigid_utils as ru
import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs
from gudiff_model.Data_Graph_Null import  Make_nullKNN_MP_Graphs
from data_rigid_diffuser import diffuser

In [4]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [5]:
from se3_transformer.model.FAPE_Loss import FAPE_loss_null, FAPE_loss_real
from se3_transformer.model.FAPE_Loss import get_t

In [8]:
# B = 16
# L=65
# limit = 1028
# h4_trainData = Helix4_Dataset(coords_tog[:limit])
# h4_valData = Helix4_Dataset(coords_apa[:limit])
# train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
# val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)
# testiter = iter(train_dL)
# bb_dict = next(testiter)



In [10]:
#I think I adjusted inputs and outputs,
#sigmoid for nodes features for real/null pred?
#need loss function for real/null nodes
#need to get function to just pull real nodes for viewing?

In [13]:
def convert_pV_to_points(dict_in):
    
    CA_fp  = dict_in['bb_firstp']['CA'].reshape(B, L, 3).to(device)
    NC_fp = CA_fp + dict_in['bb_firstp']['N_CA'].reshape(B, L, 3).to(device)
    CC_fp = CA_fp + dict_in['bb_firstp']['C_CA'].reshape(B, L, 3).to(device)
    fp =  torch.cat((NC_fp,CA_fp,CC_fp),dim=2).reshape(B,L,3,3)
    
    CA_lp  = dict_in['bb_firstp']['CA'].reshape(B, L, 3).to(device)
    NC_lp = CA_fp + dict_in['bb_firstp']['N_CA'].reshape(B, L, 3).to(device)
    CC_lp = CA_fp + dict_in['bb_firstp']['C_CA'].reshape(B, L, 3).to(device)
    lp =  torch.cat((NC_lp,CA_lp,CC_lp),dim=2).reshape(B,L,3,3)
    
    return fp, lp
def get_noise_pred_true_null(noised_dict, batched_t, graph_maker, graph_unet):
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    feat_dict = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(feat_dict, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    nf_pred = out['0']
    
    nf_pred = out['0']
    real_nodes_pred = torch.round(nf_pred ).clamp(0,1)
    real_nodes_pred_mask = (real_nodes_pred.squeeze().sum(-1)>1.99).reshape(B,L)
    
    real_nodes_true_mask = noised_dict['real_nodes_mask']
    #place roll here later
#     true = true.to('cpu').numpy()*10
#     noise_xyz = noise_xyz.to('cpu').numpy()*10
#     pred = pred.detach().to('cpu').numpy()*10
    
    
    return true, noise_xyz, pred , real_nodes_pred_mask, real_nodes_true_mask
        
def roll2_continous_true(real_mask_in):
    """Return roll amount to set zero on Nterminal residue for pdb file view"""

    roll_con_out = []
    for i,rmr in enumerate(real_mask_in):
        ep_bool = (rmr^rmr.roll(-1) | rmr^rmr.roll(1)) & rmr
        si = torch.arange(ep_bool.shape[0])[ep_bool]
        #circular if start/end real nodes and we need to roll
        if rmr[0] and rmr[-1]:
            #roll last group across barrier
            roll_con = -si[-1]
        elif not rmr[0]: #move first group to front
            roll_con = -si[0]
        else:
            roll_con=0

        roll_con_out.append(roll_con)

    return roll_con_out
      
def dump_tnp_null(true, noise, pred, t_val, e=0, numOut=1, real_mask=None, pred_mask=None, outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    tnk_dir = f'{outdir}/true_node_mask/'
    pnk_dir = f'{outdir}/pred_node_mask/'
    f_dir = f'{outdir}/full/'
    
    if not os.path.isdir(tnk_dir) and real_mask is not None:
        os.makedirs(tnk_dir)
    if not os.path.isdir(pnk_dir) and pred_mask is not None:
        os.makedirs(pnk_dir)
    if not os.path.isdir(f_dir) and real_mask is not None:
        os.makedirs(f_dir)
    
    if real_mask is not None:
        rc = roll2_continous_true(real_mask)
        for x in range(numOut):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            dump_coord_pdb(t_o, fileOut=f'{f_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o, fileOut=f'{f_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o, fileOut=f'{f_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
    if pred_mask is not None:
        rc = roll2_continous_true(pred_mask)
        for x,c in enumerate(np.arange(numOut)):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            pm = pred_mask[x].roll(int(rc[x]),dims=0)
            dump_coord_pdb(t_o[pm], fileOut=f'{pnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o[pm], fileOut=f'{pnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o[pm], fileOut=f'{pnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            
    if real_mask is not None:
        rc = roll2_continous_true(real_mask)
        for x,c in enumerate(np.arange(numOut)):
            t_o = true[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            n_o = noise[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            p_o = pred[x].roll(int(rc[x]),dims=0).detach().to('cpu').numpy()*10
            rm = real_mask[x].roll(int(rc[x]),dims=0)
            dump_coord_pdb(t_o[rm], fileOut=f'{pnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(n_o[rm], fileOut=f'{pnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            dump_coord_pdb(p_o[rm], fileOut=f'{pnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
            
#     if real_mask is not None:
#         rc = roll2_continous_true(real_mask)
#         for x in range(numOut):
#             dump_coord_pdb(true[x][real_mask[x]], fileOut=f'{tnk_dir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
#             dump_coord_pdb(noise[x][real_mask[x]], fileOut=f'{tnk_dir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
#             dump_coord_pdb(pred[x][real_mask[x]], fileOut=f'{tnk_dir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
    
    

In [14]:
def convert_pV_to_points(dict_in):
    
    CA_fp  = dict_in['bb_firstp']['CA'].to(device)
    NC_fp = CA_fp + dict_in['bb_firstp']['N_CA'].to(device)
    CC_fp = CA_fp +dict_in['bb_firstp']['C_CA'].to(device)
    fp =  torch.cat((NC_fp,CA_fp,CC_fp),dim=2).reshape(B,1,3,3)
    
    CA_lp  = dict_in['bb_lastp']['CA'].to(device)
    NC_lp = CA_fp + dict_in['bb_lastp']['N_CA'].to(device)
    CC_lp = CA_fp + dict_in['bb_lastp']['C_CA'].to(device)
    lp =  torch.cat((NC_lp,CA_lp,CC_lp),dim=2).reshape(B,1,3,3)
    
    return fp, lp
    

In [15]:


def model_step_null(noised_dict, batched_t, graph_maker, graph_unet, train=True):
    #prep coordinates for output display from and comparison via FAPE
    
    CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to(device)
    NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to(device)#not mult by bond distance, seems to help?
    CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to(device)#not mult 
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

    CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to(device)
    NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device)
    CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

    #prepare graphs
    feat_dict = graph_maker.prep_for_network(noised_dict, cuda=True)
    out =graph_unet(feat_dict,batched_t)
    
    
    #FAPE Loss for the prediction
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation , convert from x,y,z (Quat) to rotate input vectors
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device),
                            noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)),dim=2).reshape(B,L,2,1,3)
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #comparable but seems better not have it for true, but have it for pred
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #maybe this helep prevent 

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    #divide loss by real and null nodes
    
    fp, lp  = convert_pV_to_points(noised_dict)

    real_mask = noised_dict['real_nodes_mask'].to('cuda')
    score_scales = noised_dict['score_scales'].to('cuda')

    lr, lr_d = FAPE_loss_real(pred, true, score_scales, real_mask,  d_clamp=10.0, d_clamp_inter=30.0,
                   A=10.0, gamma=1.0, eps=1e-6)
    
    ln, ln_d = FAPE_loss_null(pred, fp, lp, real_mask, score_scales,  d_clamp=10.0,
                       d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)
    
    structure_loss = lr*score_weights['3D_real']+ln*score_weights['3D_null']
    
    #score for node feats determining whether node is real or fake
    nf_pred = out['0']

    nf_feat_dim = noised_dict['real_nodes_noise'].shape[-1]
    nf_true = torch.ones(noised_dict['real_nodes_mask'].shape+(nf_feat_dim,) + (1,),
                         dtype=torch.float,device=device)

    nf_real_mask_mult = real_mask.unsqueeze(-1).unsqueeze(-1).to(device)
    nf_true = nf_true*nf_real_mask_mult

    nf_pred = nf_pred.reshape(B,-1,nf_feat_dim)
    pred_nf_loss = torch.sum(torch.abs(nf_true.squeeze()-nf_pred),dim=-1) #absolute value loss
    pred_nf_loss = pred_nf_loss.to(device)
    
    ss_scales = to_cuda(noised_dict['score_scales'])[:,None,None]
    pnfloss = (torch.sum((pred_nf_loss*ss_scales/L)))*score_weights['nf_real']
    
    final_loss = structure_loss + pnfloss
    
    
    return final_loss, pnfloss.detach().cpu(), structure_loss.detach().cpu(), lr.detach().cpu(), ln.detach().cpu()



In [16]:
device='cuda'

B = 8
L= 128
limit = 1028
prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=L,
              n_atoms=5, coord_div=10, cast_type=torch.float32)
train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)
stride=4##########
mkg = Make_nullKNN_MP_Graphs(KNN=30, mp_stride=stride, n_nodes=L)

score_weights = {}
score_weights['nf_real'] = torch.tensor(0.2,device=device)
score_weights['3D_real'] = torch.tensor(1.0,device=device)
score_weights['3D_null'] = torch.tensor(1.0,device=device)

config_path='data_rigid_diffuser/base.yaml'
fnd = FrameDiffNoise(config_path)


In [17]:
gunn= GraphUNet_Null(fiber_start = Fiber({0:17, 1:2}),
                     fiber_out = Fiber({0:5,1:2}),
                      k=4,
                      batch_size = B,
                      stride=stride,
                       max_degree=3,
                       channels=64,
                      num_heads = 16,
                      channels_div=8,
                      num_layers = 1,
                     num_layers_ca = 2,
                     edge_feature_dim=1,
                     latent_pool_type = 'max',
                     t_size = 12,
                     mult=2,
                    zero_lin=True,
                   use_tdeg1 = False,
                 cuda=True).to('cuda')

opti = torch.optim.Adam(gunn.parameters(), lr=0.0005, weight_decay=5e-6)
#opti =torch.optim.Adadelta(gunn.parameters(), lr=0.0005, weight_decay=5e-7)

In [18]:
torch.optim.Adadelta

torch.optim.adadelta.Adadelta

In [19]:
testiter = iter(train_dL)
bb_dict = next(testiter)

In [20]:
t_cpu = np.ones((B,))*0.01
noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
batched_t = to_cuda(noised_dict['t_vec'])

In [21]:
t=0.02
for e in range(100):
    pnf_Score=0
    e_score = 0
    r_score = 0
    l_score = 0
    for i, bb_dict in enumerate(train_dL):
        t_cpu = np.ones((B,))*t
        
        noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
        batched_t = to_cuda(noised_dict['t_vec'])


        #final_loss, pred_nf_loss, loss_3D = model_step_null(noised_dict,batched_t, mkg, gunn, train=True)
        final_loss, pnfloss, structure_loss, real_loss, null_loss = model_step_null(noised_dict,batched_t, mkg, gunn, train=True)
        
        opti.zero_grad()
        final_loss.backward()
        opti.step()
        fl = final_loss.detach()
        e_score += fl
        pnf_Score += pnfloss
        r_score += real_loss
        l_score += null_loss
    #print(e,e_score,pred_nf_loss.sum(), loss_3D.sum())
    if e%5==1:
        true, noise_xyz, pred , real_nodes_pred_mask, real_nodes_true_mask = get_noise_pred_true_null(noised_dict, batched_t, mkg, gunn)
        dump_tnp_null(true, noise_xyz, pred, t_cpu,e=e, numOut=1, real_mask=real_nodes_true_mask, pred_mask=None, outdir='output/')
        
    print(e,e_score,pnf_Score,r_score,l_score)

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


0 tensor(3221.5481, device='cuda:0') tensor(2649.7085) tensor(443.5340) tensor(128.3047)
1 tensor(721.7962, device='cuda:0') tensor(187.8804) tensor(426.0406) tensor(107.8751)
2 tensor(524.7708, device='cuda:0') tensor(6.3072) tensor(419.9707) tensor(98.4929)
3 tensor(509.0513, device='cuda:0') tensor(4.0618) tensor(413.1630) tensor(91.8263)
4 tensor(499.3692, device='cuda:0') tensor(1.8367) tensor(408.9172) tensor(88.6152)
5 tensor(495.5051, device='cuda:0') tensor(0.8494) tensor(407.3964) tensor(87.2594)
6 tensor(496.9202, device='cuda:0') tensor(1.4569) tensor(408.6588) tensor(86.8044)
7 tensor(493.8044, device='cuda:0') tensor(0.7936) tensor(407.4102) tensor(85.6007)
8 tensor(491.7913, device='cuda:0') tensor(1.2068) tensor(406.0603) tensor(84.5241)
9 tensor(488.5466, device='cuda:0') tensor(1.5562) tensor(403.9198) tensor(83.0707)
10 tensor(484.4487, device='cuda:0') tensor(0.5390) tensor(403.2652) tensor(80.6445)
11 tensor(481.1455, device='cuda:0') tensor(0.4078) tensor(401.9539

97 tensor(347.9015, device='cuda:0') tensor(0.0001) tensor(307.3578) tensor(40.5437)
98 tensor(347.9319, device='cuda:0') tensor(0.0002) tensor(306.9611) tensor(40.9706)
99 tensor(345.1673, device='cuda:0') tensor(0.0002) tensor(305.2675) tensor(39.8996)


In [ ]:
for i, bb_dict in enumerate(train_dL):
    t_cpu = np.ones((B,))*0.05
    batched_t = to_cuda(noised_dict['t_vec'])
    noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
    break


In [ ]:
feat_dict = mkg.prep_for_network(noised_dict)
out = gunn(feat_dict, batched_t)

In [ ]:
noised_dict['real_nodes_noise'].shape

In [ ]:
#final_loss, pred_nf_loss, loss_3D = model_step_null(noised_dict, batched_t, mkg, gunn, train=True)
#final_loss, pred_nf_loss, loss_3D = model_step_null(noised_dict, batched_t, mkg, gunn, train=True)
true, noise_xyz, pred , real_nodes_pred_mask, real_nodes_true_mask = get_noise_pred_true_null(noised_dict, batched_t, mkg, gunn)
dump_tnp_null(true, noise_xyz, pred, t_cpu, numOut=1, real_mask=real_nodes_true_mask, pred_mask=None, outdir='output/')

In [ ]:
# device='cuda'

# B = 8
# L= 128
# limit = 1028
# prot_trainData = Data_Graph.ProteinBB_Dataset(coords_tog[:limit], n_nodes=L,
#               n_atoms=5, coord_div=10, cast_type=torch.float32)
# train_dL = DataLoader(prot_trainData, batch_size=B, shuffle=True, drop_last=True)
# stride=4##########
# mkg = Make_nullKNN_MP_Graphs(KNN=30, mp_stride=stride, n_nodes=L)

# score_weights = {}
# score_weights['nf_real'] = torch.tensor(0.2,device=device)
# score_weights['3D_real'] = torch.tensor(1.0,device=device)
# score_weights['3D_null'] = torch.tensor(1.0,device=device)

# config_path='data_rigid_diffuser/base.yaml'
# fnd = FrameDiffNoise(config_path)

# gunn= GraphUNet_Null(fiber_start = Fiber({0:17, 1:2}),
#                      fiber_out = Fiber({0:5,1:2}),
#                       k=4,
#                       batch_size = B,
#                       stride=stride,
#                        max_degree=3,
#                        channels=64,
#                       num_heads = 16,
#                       channels_div=8,
#                       num_layers = 1,
#                      num_layers_ca = 2,
#                      edge_feature_dim=1,
#                      latent_pool_type = 'max',
#                      t_size = 12,
#                      mult=2,
#                     zero_lin=True,
#                    use_tdeg1 = False,
#                  cuda=True).to('cuda')

# opti = torch.optim.Adam(gunn.parameters(), lr=0.0005, weight_decay=5e-6)
# #opti =torch.optim.Adadelta(gunn.parameters(), lr=0.0005, weight_decay=5e-7)


In [21]:
# testiter = iter(train_dL)
# bb_dict = next(testiter)

# t_cpu = np.ones((B,))*0.05
# noised_dict = fnd.forward(bb_dict,t_vec=t_cpu)
# batched_t = to_cuda(noised_dict['t_vec'])
# CA_t  = noised_dict['bb_shifted']['CA'].reshape(B, L, 3).to(device)
# NC_t = CA_t + noised_dict['bb_shifted']['N_CA'].reshape(B, L, 3).to(device)#not mult by bond distance, seems to help?
# CC_t = CA_t + noised_dict['bb_shifted']['C_CA'].reshape(B, L, 3).to(device)#not mult 
# true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

# CA_n  = noised_dict['bb_noised']['CA'].reshape(B, L, 3).to(device)
# NC_n = CA_n + noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device)
# CC_n = CA_n + noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)
# noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

# #prepare graphs
# feat_dict = mkg.prep_for_network(noised_dict, cuda=True)
# out =gunn(feat_dict,batched_t)


# #FAPE Loss for the prediction
# CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
# Qs = out['1'][:,1,:] # rotation , convert from x,y,z (Quat) to rotate input vectors
# Qs = Qs.unsqueeze(1).repeat((1,2,1))
# Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
# Qs = normQ(Qs)
# Rs = Qs2Rs(Qs)
# N_C_to_Rot = torch.cat((noised_dict['bb_noised']['N_CA'].reshape(B, L, 3).to(device),
#                         noised_dict['bb_noised']['C_CA'].reshape(B, L, 3).to(device)),dim=2).reshape(B,L,2,1,3)
# rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
# NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #comparable but seems better not have it for true, but have it for pred
# CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #maybe this helep prevent 

# pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)

# #divide loss by real and null nodes

# fp, lp  = convert_pV_to_points(noised_dict)

# real_mask = noised_dict['real_nodes_mask'].to('cuda')
# score_scales = noised_dict['score_scales'].to('cuda')

# lr, lr_d = FAPE_loss_real(pred, true, score_scales, real_mask,  d_clamp=10.0, d_clamp_inter=30.0,
#                A=10.0, gamma=1.0, eps=1e-6)
# ln, ln_d = FAPE_loss_null(pred, fp, lp, real_mask, true, score_scales,  d_clamp=10.0,
#                    d_clamp_inter=30.0, A=10.0, gamma=0.1, eps=1e-6)
# ld, ln_d = FAPE_loss(pred.unsqueeze(0), true,score_scales,  d_clamp=10.0,
#                    d_clamp_inter=30.0, A=10.0, gamma=1.0, eps=1e-6)